In [ ]:
from pystac import Item
from odc.stac import load
import folium
from ipyleaflet import basemaps

In [ ]:
row = 64
col = 33
year = 2024

url = (
    "https://dep-public-staging.s3.us-west-2.amazonaws.com/dep_s2_seagrass/0-0-1/"
    f"{row:03}/{col:03}/{year}/dep_s2_seagrass_{row:03}_{col:03}_{year}.stac-item.json"
)
item = Item.from_file(url)

data = load([item])

data

In [ ]:
center = data.odc.geobox.geographic_extent.centroid.coords[0][::-1]

m = folium.Map(location=center, zoom_start=10, tiles=basemaps.Esri.WorldImagery)

for variable in data.data_vars:
    data[variable].odc.add_to(m, name=variable, colormap="viridis")

folium.LayerControl().add_to(m)

m